# Exercici: Rescat en un Parc Nacional

Un excursionista s'ha perdut en un parc nacional i un equip de rescat ha d'arribar fins a la seva ubicació de la manera més eficient possible. El parc està dividit en diversos punts de referència, i cada camí entre punts té un cost associat, determinat per factors com la distància, el tipus de terreny i el clima.

## Objectiu
Trobar el camí de menor cost entre el punt d'inici del rescat i la ubicació de l'excursionista.

## Requisits
1. **Definir el graf**: Cada punt de referència és un node, i cada camí entre dos punts és una aresta amb un cost associat.
2. **Implementar una cerca de cost uniforme o informada** (per exemple, A*) per trobar el camí òptim.
3. Comparar els resultats de diferents algoritmes de cerca, com ara la cerca en amplitud, la cerca de cost uniforme, i l'algoritme A*.

## Descripció del Graf
Per simplificar, definim els nodes del graf com a lletres (per exemple, "A", "B", "C", etc.) i assignem un cost a cada camí. A continuació, es mostra el graf inicial en Python:

In [2]:
# Definició del graf amb costos d'acció
# Cada clau representa un node i el seu valor és un diccionari amb els nodes veïns i el cost per arribar-hi

graf = {
    "A": {"B": 4, "C": 3},
    "B": {"A": 4, "D": 5, "E": 12},
    "C": {"A": 3, "F": 7},
    "D": {"B": 5, "E": 4, "G": 10},
    "E": {"B": 12, "D": 4, "G": 2, "H": 5},
    "F": {"C": 7, "H": 6},
    "G": {"D": 10, "E": 2, "I": 1},
    "H": {"E": 5, "F": 6, "I": 3},
    "I": {"G": 1, "H": 3}
}

In [16]:
import queue

def ucs(graf, inici, final):
    visitats = set()
    frontera = queue.PriorityQueue()
    frontera.put((0, inici, [inici]))

    while not frontera.empty():
        cost_actual, node, cami = frontera.get()
        print(cami)
        
        visitats.add(node)

        if node == final:
            return cost_actual, final, cami
        
        for desti, cost in graf[node].items():
            if desti not in visitats:
                frontera.put((cost_actual + cost, desti, cami + [desti]))

    return None

cost, desti, cami = ucs(graf, "A", "I")
cami

['A']
['A', 'C']
['A', 'B']
['A', 'B', 'D']
['A', 'C', 'F']
['A', 'B', 'D', 'E']
['A', 'B', 'D', 'E', 'G']
['A', 'B', 'E']
['A', 'C', 'F', 'H']
['A', 'B', 'D', 'E', 'G', 'I']


['A', 'B', 'D', 'E', 'G', 'I']

In [ ]:
from dataclasses import dataclass

@dataclass
class Estat:
    node: str = ""
    pare: 'Estat' = None

    def __lt__(self, other):
        return self.node < other.node
        

def ucs_estats(graf, inici, final):
    estat_inicial = Estat(inici, None)
    visitats = set()
    frontera = queue.PriorityQueue()

    frontera.put((0, estat_inicial))

    while not frontera.empty():
        cost_actual, estat = frontera.get()
        
        node = estat.node
        visitats.add(estat.node)

        if node == final:
            return cost_actual, estat

        for desti, cost in graf[node].items():
            if desti not in visitats:
                nou_estat = Estat(desti, estat)

                frontera.put((cost_actual + cost, nou_estat))

    return None

cost_actual, estat = ucs_estats(graf, "A", "I")

while estat.pare is not None:
    print(estat.node)
    estat = estat.pare


I
G
E
D
B


## Tasques

- **Implementar l'algoritme de cerca** que considerem adequat per trobar el camí de menor cost des de l’inici fins a la destinació.
- **Calcular i mostra el camí i el cost total** del recorregut trobat.
- **Analitzar els resultats**: Compara la ruta obtinguda amb diferents tipus de cerca, si en proves més d'una (per exemple, cerca de cost uniforme i A*).

### Punt de Partida i Destinació

- **Punt d'inici**: Node "A"
- **Punt de destinació** (ubicació de l'excursionista): Node "I"

## Solució

En primer lloc implementarem l'algorisme UCS (Uniform Cost Search) per trobar el camí de menor cost entre el punt d'inici i el punt de destinació. De moment no ens preocuparem de reconstruir el camí, simplement ens centrarem en trobar el cost total.

In [9]:
import queue

def ucs(graf, node_inicial, node_final):
    # Inicialitzem la llista de nodes visitats
    visitats = set()
    # Inicialitzem la llista de nodes per visitar
    frontera = queue.PriorityQueue()
    # Inicialitzem la frontera amb el node inicial
    frontera.put((0, node_inicial))
    # Per poder comparar els estats avaluats incialitzem avaluats a 0
    avaluats = 0

    # Mentre la frontera no estigui buida
    while not frontera.empty():
        # Treiem el node de la frontera
        (cost, node) = frontera.get()
        # Incrementem el nombre d'estats avaluats
        avaluats += 1

        # Si el node no ha estat visitat
        if node[-1] not in visitats:
            # Marquem el node com a visitat
            visitats.add(node)

            # Si el node és el node final, retornem el camí
            if node == node_final:
                return node, cost, avaluats

            # Afegim els veïns del node a la frontera, cost = ucs(graf, node_inicial, node_final)
            for vei, cost_vei in graf[node[-1]].items():
                if vei not in visitats:
                    frontera.put((cost + cost_vei, vei))

    # Si no hi ha camí, retornem None
    return None

# Exemple d'ús
node_inicial = "A"
node_final = "I"

final, cost, avaluats = ucs(graf, node_inicial, node_final)
print("Cost:", cost)
print("Avaluats:", avaluats)

Cost: 16
Avaluats: 10


Per a poder reconstruir el camí, necessitarem modificar l'algorisme UCS perquè ens retorni el camí complet, no només el cost.

In [10]:
def ucs_ruta(graf, inici, desti):
    # Inicialitzem la llista de nodes visitats
    visitats = set()
    # Inicialitzem la llista de nodes per visitar
    frontera = queue.PriorityQueue()
    # Inicialitzem la frontera amb el node inicial, cost = 0 i camí = [node_inicial]
    frontera.put((0, [node_inicial], [node_inicial]))
    # Per poder comparar els estats avaluats incialitzem avaluats a 0
    avaluats = 0

    # Mentre la frontera no estigui buida
    while not frontera.empty():
        # Treiem el node de la frontera
        cost, node, cami = frontera.get()
        # Incrementem el nombre d'estats avaluats
        avaluats += 1

        # Si el node no ha estat visitat
        if node not in visitats:
            # Marquem el node com a visitat
            visitats.add(node)

            # Si el node és el node final, retornem el camí
            if node == desti:
                return cami, cost, avaluats, cami

            # Afegim els veïns del node a la frontera, cost = ucs(graf, node_inicial, node_final)
            for vei, cost_vei in graf[node].items():
                if vei not in visitats:
                    frontera.put((cost + cost_vei, vei, cami + [vei]))

    # Si no hi ha camí, retornem None
    return None
cost
# Exemple d'ús
node_inicial = "A"
node_final = "I"

final, cost, avaluats, cami = ucs_ruta(graf, node_inicial, node_final)


TypeError: unhashable type: 'list'